<a href="https://colab.research.google.com/github/akshaygrao77/DeepLearning-Assignment2/blob/main/PART_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import and install required libraries
import os
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout, BatchNormalization
import numpy as np

!pip install wandb
import wandb
from wandb.keras import WandbCallback

from PIL import Image

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

     |████████████████████████████████| 1.7 MB 22.0 MB/s 
     |████████████████████████████████| 144 kB 37.6 MB/s 
     |████████████████████████████████| 181 kB 36.6 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=3c31673fd5e7fbf52ccb3e5b290d48e6de44294450f5677a7871843964ebd9d2
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Unzip and load file data file onto server, then delete zip file for optimizing performance
zip_path = "drive/MyDrive/nature_12K.zip"
!cp "{zip_path}" .
!unzip -q nature_12K.zip
!rm nature_12K.zip

In [ ]:
##  building of CNN Model
def CNN_Model(num_filters=32, filter_multiplier=1, dropout=0.2, batch_norm=False, dense_size=64, num_classes=10, image_size=200):
    cnn_model = Sequential()
    
    for i in range(5):
        filter_dim = 11 - 2*i
        filter_size = (filter_dim, filter_dim)
        if i==0:
            cnn_model.add(Conv2D(num_filters, filter_size, input_shape=(image_size, image_size, 3), data_format="channels_last"))
        else:
            cnn_model.add(Conv2D(num_filters, filter_size))
        if batch_norm:
            cnn_model.add(BatchNormalization())
        cnn_model.add(Activation("relu"))
        cnn_model.add(MaxPooling2D(pool_size=(2,2)))
        num_filters = int(num_filters * filter_multiplier)
    
    cnn_model.add(Flatten())
    cnn_model.add(Dense(dense_size))
    cnn_model.add(Dropout(dropout))
    cnn_model.add(Activation("relu"))
    cnn_model.add(Dense(num_classes))
    cnn_model.add(Activation("softmax"))

    return cnn_model